In [ ]:
import mne
import glob
import os
import sys
import logging
import numpy as np
import pandas as pd

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.model_selection import GridSearchCV
from scipy.signal import find_peaks
from scipy.stats import skew, kurtosis
from sklearn.metrics import make_scorer, accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import confusion_matrix

## Helper Functions

In [ ]:
def split_train_test_path_list(data_path, file_name_template, train_ratio):
    file_list = sorted(glob.glob(os.path.join(data_path, file_name_template)))
    np.random.shuffle(file_list)
    split_id = int(len(file_list) * train_ratio)

    train_list = file_list[:split_id]
    test_list = file_list[split_id:]

    return train_list, test_list

def read_eeg_epochs(train_list, test_list):
    epochs_train_list = []
    epochs_test_list = []

    for file_path in train_list:
        with mne.utils.use_log_level("ERROR"):
            epoch_train = mne.read_epochs(file_path, preload=True)
            epochs_train_list.append(epoch_train)

    for file_path in test_list:
        with mne.utils.use_log_level("ERROR"):
            epoch_test = mne.read_epochs(file_path, preload=True)
            epochs_test_list.append(epoch_test)

    epochs_train = mne.concatenate_epochs(epochs_train_list)
    epochs_test = mne.concatenate_epochs(epochs_test_list)

    return epochs_train, epochs_test


def get_window_idx(center, width, times_arr):
    return np.where((times_arr >= center - width/2) & (times_arr <= center + width/2))[0]

def zero_crossings(sig):
    return np.where(np.diff(np.sign(sig)))[0].size

def extract_peak_features(X, times, window_size=0.04, baseline_correction = False):

    n_epochs, n_channels, n_times = X.shape
    features = []

    peak_window_N170 =(0.12, 0.22)
    peak_window_P100=(0.05, 0.15)
    peak_window_P300 = (0.2, 0.3)

    win_mask_N170 = (times >= peak_window_N170[0]) & (times <= peak_window_N170[1])
    win_mask_P100 = (times >= peak_window_P100[0]) & (times <= peak_window_P100[1])
    win_mask_P300 = (times >= peak_window_P300[0]) & (times <= peak_window_P300[1])

    for ep in range(n_epochs):
        feats_ep = []
        for ch in range(n_channels):
            signal = X[ep, ch, :]


            sig_win_P100 = signal[win_mask_P100]
            times_win_P100 = times[win_mask_P100]

            peaks, _ = find_peaks(sig_win_P100, distance=100)
            if len(peaks) == 0:
                idx1 = np.argmax(sig_win_P100)
            else:
                idx1 = peaks[0]
            time1 = times_win_P100[idx1]
            amp1 = float(sig_win_P100[idx1])
            if baseline_correction:
                amp1 -= float(sig_win_P100[0])


            sig_win_N170 = signal[win_mask_N170]
            times_win_N170 = times[win_mask_N170]

            peaks_min, _ = find_peaks(-sig_win_N170, distance=100)
            if len(peaks_min) == 0:
                idx2 = np.argmin(sig_win_N170)
            else:
                idx2 = peaks_min[0]
            time2 = times_win_N170[idx2]
            amp2 = float(sig_win_N170[idx2])
            if baseline_correction:
                amp2 -= float(sig_win_N170[0])


            sig_win_P300 = signal[win_mask_P300]
            times_win_P300 = times[win_mask_P300]

            peaks, _ = find_peaks(sig_win_P300, distance=100)
            if len(peaks) == 0:
                idx3 = np.argmax(sig_win_P300)
            else:
                idx3 = peaks[0]
            time3 = times_win_P300[idx3]
            amp3 = float(sig_win_P300[idx3])
            if baseline_correction:
                amp3 -= float(sig_win_P300[0])

            win1 = get_window_idx(time1, window_size, times)
            win2 = get_window_idx(time2, window_size, times)
            win3 = get_window_idx(time3, window_size, times)

            feats_ep.extend([time1, time2, time3, amp1, amp2, amp3])

            for win in [win1, win2, win3]:
                if len(win) > 0:
                    sig_win = signal[win]
                    t_win = times[win]

                    # 7. Zero-crossing rate
                    feat_zc = zero_crossings(sig_win)
                    # 8. Peak-to-peak amplitude
                    feat_ptp = np.ptp(sig_win)

                    if baseline_correction:
                        sig_win = sig_win - sig_win[0]

                    # Feature calculations:
                    # 1. RMS
                    feat_rms = np.sqrt(np.mean(sig_win ** 2))
                    # 2. Variance
                    feat_var = np.var(sig_win)
                    # 3. Std
                    feat_std = np.std(sig_win)
                    # 4. Skewness
                    feat_skew = skew(sig_win)
                    # 5. Kurtosis
                    feat_kurt = kurtosis(sig_win)
                    # 6. Area under the curve
                    feat_auc = np.sum(sig_win)
                    # 9. Slope
                    # Fit line: polyfit returns [slope, intercept]
                    slope = np.polyfit(t_win, sig_win, 1)[0] if len(sig_win) > 1 else np.nan
                    # 10. Mean
                    feat_mean = np.mean(sig_win)
                    # 11. Min
                    feat_min = np.min(sig_win)
                    # 12. Max
                    feat_max = np.max(sig_win)
                    # 13. Median
                    feat_median = np.median(sig_win)

                    feats_ep.extend([
                        feat_rms, feat_var, feat_std, feat_skew, feat_kurt, feat_auc, feat_zc, feat_ptp, slope, feat_mean, feat_min, feat_max, feat_median
                    ])
                else:
                    feats_ep.extend([np.nan]*14)

        features.append(feats_ep)
    return np.array(features)  # shape: (n_epochs, n_channels*3*14)


def get_X_and_Y_from_epochs(train_list, test_list, events, picks=None, t_min = -0.2, t_max = 0.5):

    epochs_train, epochs_test = read_eeg_epochs(train_list, test_list)

    #####---------------------------------------------------------------------------------------------------------

    epochs_train_list_event1 = epochs_train[events[0]].get_data(picks=picks, tmin=t_min, tmax=t_max)
    epochs_train_list_event2 = epochs_train[events[1]].get_data(picks=picks, tmin=t_min, tmax=t_max)
    X_train = np.concatenate((epochs_train_list_event1, epochs_train_list_event2), axis=0)

    labels_train_event1 = [0] * len(epochs_train_list_event1)
    labels_train_event2 = [1] * len(epochs_train_list_event2)
    y_train = np.concatenate((labels_train_event1, labels_train_event2), axis=0)

    ######--------------------------------------------------------------------------------------------------------

    epochs_test_list_event1 = epochs_test[events[0]].get_data(picks=picks, tmin=t_min, tmax=t_max)
    epochs_test_list_event2 = epochs_test[events[1]].get_data(picks=picks, tmin=t_min, tmax=t_max)
    X_test = np.concatenate((epochs_test_list_event1, epochs_test_list_event2), axis=0)

    labels_test_event1 = [0] * len(epochs_test_list_event1)
    labels_test_event2 = [1] * len(epochs_test_list_event2)
    y_test = np.concatenate((labels_test_event1, labels_test_event2), axis=0)

    return X_train, X_test, y_train, y_test


def get_X_and_Y_from_epochs_with_feature_extraction(train_list, test_list, events, picks=None, window_size = 0.02):

    t_min = -0.2
    t_max = 0.5

    X_train, X_test, y_train, y_test = get_X_and_Y_from_epochs(train_list, test_list, events, picks, t_min, t_max)

    times = np.linspace(t_min, t_max, X_train.shape[2])

    # Feature exctraction from peaks
    X_train_feats = extract_peak_features(X_train, times,window_size)
    X_test_feats = extract_peak_features(X_test, times, window_size)

    logging.info(f"shape: {X_train_feats.shape}")

    return X_train_feats, X_test_feats, y_train, y_test

def eval_split(name, X, y, model, condition_in_out):
    y_pred = model.predict(X)
    cm = confusion_matrix(y, y_pred)

    logging.info(f"\n== {name.upper()} ==")
    logging.info(f"AUC      : {roc_auc_score(y, model.predict_proba(X)[:, 1]):.4f}")
    logging.info(f"Accuracy : {accuracy_score(y, y_pred):.4f}")
    logging.info(f"F1       : {f1_score(y, y_pred, pos_label=1):.4f}")
    logging.info(f"Precision: {precision_score(y, y_pred, pos_label=1):.4f}")
    logging.info(f"Recall   : {recall_score(y, y_pred, pos_label=1):.4f}")
    if condition_in_out:
        cm_df = pd.DataFrame(cm, index=["Actual in-group", "Actual out-group"], columns=["Predicted in-group", "Predicted out-group"])
    else:
        cm_df = pd.DataFrame(cm, index=["Actual inv", "Actual up"], columns=["Predicted inv", "Predicted up"])
    logging.info(f"\nConfusion matrix ({name}):")
    logging.info(cm_df)


def train_and_test_model(X_train, X_test, y_train, y_test, model, name, condition_in_out):

    logging.info(f"\n== {name.upper()} ==")
    model.fit(X_train, y_train)

    eval_split("train", X_train, y_train, model, condition_in_out)
    eval_split("test",  X_test,  y_test, model, condition_in_out)

    logging.info("\n== GridSearchCV ==")
    logging.info(f"Best params: {model.best_params_}")

    return model

In [ ]:
dir_path = 'D:\studia\magisterka\dane EEG\BADANIE_POLITYCZNE_2022_eeg_bdfy\EEG_preprocessed'
file_name_template = "s*.bdf-epo.fif"
train_ratio = 0.8
selected_channels = ['P5', 'P6', 'P7', 'P8','PO7', 'PO8']

flatten_transformer = FunctionTransformer(lambda X: X.reshape(X.shape[0], -1))

## LOGGER

In [ ]:
log_file = "training_log.txt"

for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler(log_file, mode="a"), # a to overwrite
        logging.StreamHandler()
    ]
)

# Wyłącz logi zewnętrznych bibliotek (MNE, sklearn, matplotlib, itp.)
logging.getLogger("mne").setLevel(logging.ERROR)
logging.getLogger("sklearn").setLevel(logging.ERROR)
logging.getLogger("matplotlib").setLevel(logging.ERROR)

## GBC pipeline & param grid

In [ ]:
model_gbc = Pipeline(steps=[
    ('reshape', flatten_transformer),
    ('scaler', StandardScaler()),
    ('gbc', GradientBoostingClassifier())
])

param_grid_gbc = {
    'gbc__n_estimators': [100, 200]
}

## LDA pipeline & param grid

In [ ]:
model_lda = Pipeline(steps=[
    ('reshape', flatten_transformer),
    ('scaler', StandardScaler()),
    ('lda', LinearDiscriminantAnalysis())
])

param_grid_lda = {
    "lda__solver": ["lsqr"],
    "lda__shrinkage": [0.2, 0.5, 0.7]
}

## MODEL 1: GBC IN/OUT Exploration

In [ ]:
train_list, test_list = split_train_test_path_list(dir_path, file_name_template, train_ratio)
X_train, X_test, y_train, y_test = get_X_and_Y_from_epochs(train_list, test_list, ["in", "out"],  selected_channels, t_min=0.1, t_max=0.25)
model_1 = GridSearchCV(model_gbc, param_grid_gbc,  scoring=make_scorer(roc_auc_score, needs_threshold=True), cv=5)

train_and_test_model(X_train, X_test, y_train, y_test, model_1, "GBC IN/OUT EXPLORATION", condition_in_out = True)

## MODEL 2: GBC IN/OUT Feature Extraction

In [ ]:
model_gbc = Pipeline(steps=[
    ('reshape', flatten_transformer),
    ('scaler', StandardScaler()),
    ('gbc', GradientBoostingClassifier())
])

param_grid = {
    'gbc__n_estimators': [100, 200]
}

train_list, test_list = split_train_test_path_list(dir_path, file_name_template, train_ratio)
X_train, X_test, y_train, y_test = get_X_and_Y_from_epochs_with_feature_extraction(train_list, test_list, ["in", "out"],  selected_channels)
model_2 = GridSearchCV(model_gbc, param_grid,  scoring="roc_auc", cv=5)

train_and_test_model(X_train, X_test, y_train, y_test, model_2, "GBC IN/OUT Feature Extraction", condition_in_out = True)

## MODEL 3: LDA IN/OUT Exploration

In [ ]:
train_list, test_list = split_train_test_path_list(dir_path, file_name_template, train_ratio)
X_train, X_test, y_train, y_test = get_X_and_Y_from_epochs(train_list, test_list, ["in", "out"], selected_channels, t_min=0.1, t_max=0.25)
model_3 = GridSearchCV(model_lda, param_grid_lda,  scoring="roc_auc", cv=5)

train_and_test_model(X_train, X_test, y_train, y_test, model_3, "LDA IN/OUT Exploration", condition_in_out = True)

## MODEL 4: LDA IN/OUT Feature Extraction

In [ ]:
train_list, test_list = split_train_test_path_list(dir_path, file_name_template, train_ratio)
X_train, X_test, y_train, y_test = get_X_and_Y_from_epochs_with_feature_extraction(train_list, test_list, ["in", "out"],  selected_channels)
model_4 = GridSearchCV(model_lda, param_grid_lda,  scoring="roc_auc", cv=5)

train_and_test_model(X_train, X_test, y_train, y_test, model_4, "LDA IN/OUT Feature Extraction", condition_in_out = True)

## MODEL 5: GBC INV/UP Exploration


In [ ]:
train_list, test_list = split_train_test_path_list(dir_path, file_name_template, train_ratio)
X_train, X_test, y_train, y_test = get_X_and_Y_from_epochs(train_list, test_list, ["inv", "up"],  selected_channels, t_min=0.1, t_max=0.25)
model_5 = GridSearchCV(model_gbc, param_grid_gbc,  scoring="roc_auc", cv=5)

train_and_test_model(X_train, X_test, y_train, y_test, model_5, "GBC INV/UP EXPLORATION", condition_in_out = False)

## MODEL 6: GBC INV/UP Feature Extraction

In [ ]:
train_list, test_list = split_train_test_path_list(dir_path, file_name_template, train_ratio)
X_train, X_test, y_train, y_test = get_X_and_Y_from_epochs_with_feature_extraction(train_list, test_list, ["inv", "up"],  selected_channels)
model_6 = GridSearchCV(model_gbc, param_grid_gbc,  scoring="roc_auc", cv=5)

train_and_test_model(X_train, X_test, y_train, y_test, model_6, "GBC INV/UP Feature Extraction", condition_in_out = False)

## MODEL 7: LDA INV/UP Exploration

In [ ]:
train_list, test_list = split_train_test_path_list(dir_path, file_name_template, train_ratio)
X_train, X_test, y_train, y_test = get_X_and_Y_from_epochs(train_list, test_list, ["inv", "up"], selected_channels, t_min=0.1, t_max=0.25)
model_7 = GridSearchCV(model_lda, param_grid_lda,  scoring="roc_auc", cv=5)

train_and_test_model(X_train, X_test, y_train, y_test, model_7, "LDA INV/UP Exploration", condition_in_out = False)

## MODEL 8: LDA INV/UP Feature Extraction

In [ ]:
train_list, test_list = split_train_test_path_list(dir_path, file_name_template, train_ratio)
X_train, X_test, y_train, y_test = get_X_and_Y_from_epochs_with_feature_extraction(train_list, test_list, ["inv", "up"],
                                                                                   selected_channels)
model_8 = GridSearchCV(model_lda, param_grid_lda, scoring="roc_auc", cv=5)

train_and_test_model(X_train, X_test, y_train, y_test, model_8, "LDA INV/UP Feature Extraction", condition_in_out=False)

## Subsample average

In [ ]:
def subsample_average(data, subsample_size = 5):
    data_copy = data.copy()
    averaged_data = []

    while len(data_copy) >= subsample_size:
        indices = np.random.choice(len(data_copy), subsample_size, replace=False)

        selected = data_copy[indices]
        averaged = np.mean(selected, axis=0)
        averaged_data.append(averaged)

        mask = np.ones(len(data_copy), dtype=bool)
        mask[indices] = False
        data_copy = data_copy[mask]

    return np.array(averaged_data)


def get_X_and_Y_from_epochs_subsample_averaging(train_list, test_list, events, picks=None, t_min = -0.2, t_max = 0.5, subsample_size = 5):

    epochs_train, epochs_test = read_eeg_epochs(train_list, test_list)

    #####---------------------------------------------------------------------------------------------------------

    epochs_train_list_event1 = epochs_train[events[0]].get_data(picks=picks, tmin=t_min, tmax=t_max)
    subsample_average_train_list_event1 = subsample_average(epochs_train_list_event1, subsample_size=subsample_size)
    epochs_train_list_event2 = epochs_train[events[1]].get_data(picks=picks, tmin=t_min, tmax=t_max)
    subsample_average_train_list_event2 = subsample_average(epochs_train_list_event2, subsample_size=subsample_size)
    X_train = np.concatenate((subsample_average_train_list_event1, subsample_average_train_list_event2), axis=0)

    labels_up_train = [0] * len(subsample_average_train_list_event1)
    labels_inv_train = [1] * len(subsample_average_train_list_event2)
    y_train = np.concatenate((labels_up_train, labels_inv_train), axis=0)

    ######--------------------------------------------------------------------------------------------------------

    epochs_test_list_event1 = epochs_test[events[0]].get_data(picks=picks, tmin=t_min, tmax=t_max)
    subsample_average_test_list_event1 = subsample_average(epochs_test_list_event1, subsample_size=subsample_size)
    epochs_test_list_event2 = epochs_test[events[1]].get_data(picks=picks, tmin=t_min, tmax=t_max)
    subsample_average_test_list_event2 = subsample_average(epochs_test_list_event2, subsample_size=subsample_size)
    X_test = np.concatenate((subsample_average_test_list_event1, subsample_average_test_list_event2), axis=0)

    labels_up_test = [0] * len(subsample_average_test_list_event1)
    labels_inv_test = [1] * len(subsample_average_test_list_event2)
    y_test = np.concatenate((labels_up_test, labels_inv_test), axis=0)

    logging.info(f"shape: {X_train.shape}")


    return X_train, X_test, y_train, y_test

## GBC IN OUT EXPLORATION SUBSAMPLE AVERAGE

In [ ]:
train_list, test_list = split_train_test_path_list(dir_path, file_name_template, train_ratio)
X_train, X_test, y_train, y_test = get_X_and_Y_from_epochs_subsample_averaging(train_list, test_list, ["in", "out"],  selected_channels, t_min=0.1, t_max=0.25, subsample_size=3)
model_subsample = GridSearchCV(model_gbc, param_grid_gbc,  scoring="roc_auc", cv=5)

train_and_test_model(X_train, X_test, y_train, y_test, model_subsample, "GBC IN/OUT EXPLORATION SUBSAMPLE AVERAGE = 3", condition_in_out = True)

## GBC INV/UP EXPLORATION SUBSAMPLE AVERAGE

In [ ]:
train_list, test_list = split_train_test_path_list(dir_path, file_name_template, train_ratio)
X_train, X_test, y_train, y_test = get_X_and_Y_from_epochs_subsample_averaging(train_list, test_list, ["inv", "up"],  selected_channels, t_min=0.1, t_max=0.25, subsample_size=3)
model_subsample2 = GridSearchCV(model_gbc, param_grid_gbc, scoring="roc_auc", cv=5)

train_and_test_model(X_train, X_test, y_train, y_test, model_subsample2, "GBC INV/UP EXPLORATION SUBSAMPLE AVERAGE = 3", condition_in_out = False)

## LDA IN/OUT EXPLORATION SUBSAMPLE AVERAGE

In [ ]:
train_list, test_list = split_train_test_path_list(dir_path, file_name_template, train_ratio)
X_train, X_test, y_train, y_test = get_X_and_Y_from_epochs_subsample_averaging(train_list, test_list, ["in", "out"],
                                                                               selected_channels, t_min=0.1, t_max=0.25,
                                                                               subsample_size=5)
model_subsample3 = GridSearchCV(model_lda, param_grid_lda, scoring="roc_auc", cv=5)

train_and_test_model(X_train, X_test, y_train, y_test, model_subsample3, "LDA IN/OUT EXPLORATION SUBSAMPLE AVERAGE = 5",
                     condition_in_out=True)

## LDA INV/UP EXPLORATION SUBSAMPLE AVERAGE

In [ ]:
train_list, test_list = split_train_test_path_list(dir_path, file_name_template, train_ratio)
X_train, X_test, y_train, y_test = get_X_and_Y_from_epochs_subsample_averaging(train_list, test_list, ["inv", "up"],  selected_channels, t_min=0.1, t_max=0.25, subsample_size=3)
model_subsample4 = GridSearchCV(model_lda, param_grid_lda, scoring="roc_auc", cv=5)

train_and_test_model(X_train, X_test, y_train, y_test, model_subsample4, "LDA INV/UP EXPLORATION SUBSAMPLE AVERAGE = 5", condition_in_out = False)